In [2]:
!pip3 install sentence_transformers
!pip3 install transformers


In [3]:
import tarfile
tar = tarfile.open("yelp_dataset.tar")
tar.extractall()
tar.close()

In [ ]:
!pip install pyspark

In [4]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [5]:
spark = SparkSession.builder.appName('yelp_dataset').getOrCreate()

In [6]:
reviews_sk = spark.read.json('yelp_academic_dataset_review.json')

In [7]:
# We will get a sample of 10L from the dataset
sample_review=reviews_sk.sample(False, 0.1, seed=0).limit(1000000)
reviews_pd = sample_review.select("*").toPandas()

In [8]:
reviews_pd

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,YjUWPpI6HXG530lwP-fb2A,0,2014-02-05 20:30:30,0,saUsX_uimxRlCVr67Z4Jig,3.0,Family diner. Had the buffet. Eclectic assortm...,0,8g_iMtfSiwikVnbP2etR0A
1,gebiRewfieSdtt17PTW6Zg,0,2016-07-25 07:31:06,0,pUycOfUwM8vqX7KjRRhUEA,3.0,Had a party of 6 here for hibachi. Our waitres...,0,59MxRhNVhU9MYndMkz0wtw
2,YtSqYv1Q_pOltsVPSx54SA,0,2013-06-24 11:21:25,0,oyaMhzBSwfGgemSGuZCdwQ,5.0,Tremendous service (Big shout out to Douglas) ...,0,Dd1jQj7S-BFGqRbApFzCFw
3,x4XdNhp0Xn8lOivzc77J-g,0,2013-08-15 15:27:51,0,G_5UczbCBJriUAbxz3J7Tw,5.0,Best thai food in the area. Everything was au...,0,clWLI5OZP2ad25ugMVI8gg
4,j8JOZvfeHEfUWq3gEz6ABQ,0,2014-06-11 14:55:14,3,Ki90-hauSIfW_7sBuBBqfA,2.0,NEVER AGAIN. This is a so called restaurant th...,2,Z2cOL3n9V8NoguJ-uU_Nvw
...,...,...,...,...,...,...,...,...,...
698385,69hjJPa0gKpRQ_pxMIe_vw,0,2020-02-26 04:10:38,0,D6pl92UhO07Ew_6wg9A9zw,1.0,I visited with this doctor a couple of month a...,5,DEJSlN8SOXYvnriHansb0A
698386,pQAQwhBlSQdG1HuuLuCqXw,45,2020-10-11 00:09:30,17,kZiKvXxK7o5i7fa32u5Jgw,5.0,"Just $5 every SUNDAY in October! Do it!\n\nOh,...",46,6jjHo9Lilv3kTy87pm2ycw
698387,VOITl4HlmC1EGJ_wVMS6_w,0,2018-11-09 18:09:35,0,CphIcLTNU26TDb_YTOSgUQ,5.0,I compete in Jiu-Jitsu and every competition I...,0,Is5IWDXWtqVMCVtedjJIQQ
698388,3gVSrS4kffGGZT8oXHsIcw,2,2014-11-03 14:45:46,0,nLjbVsETpqO17RbFcqskkA,3.0,"*Later Yelp* I've only been here once, but I l...",2,am7-gkH_PDz598oTdYSD6A


In [9]:
# Overview of the converted sample
reviews_pd.to_csv('yelp_review_sample.csv')
reviews_pd.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,YjUWPpI6HXG530lwP-fb2A,0,2014-02-05 20:30:30,0,saUsX_uimxRlCVr67Z4Jig,3.0,Family diner. Had the buffet. Eclectic assortm...,0,8g_iMtfSiwikVnbP2etR0A
1,gebiRewfieSdtt17PTW6Zg,0,2016-07-25 07:31:06,0,pUycOfUwM8vqX7KjRRhUEA,3.0,Had a party of 6 here for hibachi. Our waitres...,0,59MxRhNVhU9MYndMkz0wtw
2,YtSqYv1Q_pOltsVPSx54SA,0,2013-06-24 11:21:25,0,oyaMhzBSwfGgemSGuZCdwQ,5.0,Tremendous service (Big shout out to Douglas) ...,0,Dd1jQj7S-BFGqRbApFzCFw
3,x4XdNhp0Xn8lOivzc77J-g,0,2013-08-15 15:27:51,0,G_5UczbCBJriUAbxz3J7Tw,5.0,Best thai food in the area. Everything was au...,0,clWLI5OZP2ad25ugMVI8gg
4,j8JOZvfeHEfUWq3gEz6ABQ,0,2014-06-11 14:55:14,3,Ki90-hauSIfW_7sBuBBqfA,2.0,NEVER AGAIN. This is a so called restaurant th...,2,Z2cOL3n9V8NoguJ-uU_Nvw


In [10]:
business_sk=spark.read.json('yelp_academic_dataset_business.json')
business_pd=business_sk.select("*").toPandas()

In [11]:
business_pd.columns


Index(['address', 'attributes', 'business_id', 'categories', 'city', 'hours',
       'is_open', 'latitude', 'longitude', 'name', 'postal_code',
       'review_count', 'stars', 'state'],
      dtype='object')

In [12]:
list(business_pd.categories.unique())


['Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists',
 'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services',
 'Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores',
 'Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries',
 'Brewpubs, Breweries, Food',
 'Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants',
 'Sporting Goods, Fashion, Shoe Stores, Shopping, Sports Wear, Accessories',
 'Synagogues, Religious Organizations',
 'Pubs, Restaurants, Italian, Bars, American (Traditional), Nightlife, Greek',
 'Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food',
 'Department Stores, Shopping, Fashion',
 'Vietnamese, Food, Restaurants, Food Trucks',
 'American (Traditional), Restaurants, Diners, Breakfast & Brunch',
 'General Dentistry, Dentists, Health & Medical, Cosmetic Dentists',
 'Food, Delis, Italian, Bakeries, Restaurants',
 'Sushi Bars

In [13]:

csv_tags=['Fashion','Wear','Clothing','Clothings']

In [14]:
business_pd.to_csv('yelp_business.csv')

In [15]:
# We will add a column serves_food to identify if the store serves food or not
clothes=[]
for i in range(len(business_pd)):
    try:
        if(business_pd.iloc[i]['categories'] is not None):
            categories=business_pd.iloc[i]['categories'].split(',')
            categories=[en.replace(' ','') for en in categories]
            matched_tags=(list(set(categories).intersection(csv_tags)))
            if(len(matched_tags)>0):
                clothes.append(1)
            else:
                clothes.append(0) 
        else:
            clothes.append(0)             
    except:
        print(business_pd.iloc[i]['categories'])
        clothes.append(2)                  

In [17]:
business_pd.head(10)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"1616 Chapala St, Ste 2","(None, None, None, None, None, None, None, Non...",Pns2l4eNsfO8kk83dixA6A,"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara,None,0,34.426679,-119.711197,"Abby Rappoport, LAC, CMQ",93101,7,5.0,CA
1,87 Grasso Plaza Shopping Center,"(None, None, None, None, None, None, None, Non...",mpf3x-BjTdTEA3yCZrAYPw,"Shipping Centers, Local Services, Notaries, Ma...",Affton,"(8:0-18:30, 0:0-0:0, 8:0-14:0, None, 8:0-18:30...",1,38.551126,-90.335695,The UPS Store,63123,15,3.0,MO
2,5255 E Broadway Blvd,"(None, None, None, None, None, None, None, Tru...",tUFrWirKiKi_TAnsVWINQQ,"Department Stores, Shopping, Fashion, Home & G...",Tucson,"(8:0-23:0, 8:0-22:0, 8:0-23:0, 8:0-22:0, 8:0-2...",0,32.223236,-110.880452,Target,85711,22,3.5,AZ
3,935 Race St,"(None, None, u'none', None, None, None, None, ...",MTSW4McQd7CbVtyjqoe9mw,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia,"(7:0-21:0, 7:0-20:0, 7:0-21:0, 7:0-21:0, 7:0-2...",1,39.955505,-75.155564,St Honore Pastries,19107,80,4.0,PA
4,101 Walnut St,"(None, None, None, None, None, None, None, Tru...",mWMc6_wTdE0EUBKIGXDVfA,"Brewpubs, Breweries, Food",Green Lane,"(12:0-22:0, None, 12:0-22:0, 12:0-18:0, 16:0-2...",1,40.338183,-75.471659,Perkiomen Valley Brewery,18054,13,4.5,PA
5,615 S Main St,"(None, None, u'none', None, None, None, None, ...",CF33F8-E6oudUQ46HnavjQ,"Burgers, Fast Food, Sandwiches, Food, Ice Crea...",Ashland City,"(9:0-0:0, 0:0-0:0, 9:0-22:0, 8:0-22:0, 6:0-22:...",1,36.269593,-87.058943,Sonic Drive-In,37015,6,2.0,TN
6,"8522 Eager Road, Dierbergs Brentwood Point","(None, None, None, None, None, None, None, Tru...",n_0UpQx1hsNbnPUSlodU8w,"Sporting Goods, Fashion, Shoe Stores, Shopping...",Brentwood,"(10:0-18:0, 0:0-0:0, 10:0-18:0, 12:0-18:0, 10:...",1,38.627695,-90.340465,Famous Footwear,63144,13,2.5,MO
7,400 Pasadena Ave S,None,qkRM_2X51Yqxk3btlwAQIg,"Synagogues, Religious Organizations",St. Petersburg,"(9:0-17:0, 9:0-17:0, None, None, 9:0-17:0, 9:0...",1,27.766590,-82.732983,Temple Beth-El,33707,5,3.5,FL
8,8025 Mackenzie Rd,"(None, None, u'full_bar', {'romantic': False, ...",k0hlBqXX-Bt0vf1op7Jr1w,"Pubs, Restaurants, Italian, Bars, American (Tr...",Affton,None,0,38.565165,-90.321087,Tsevi's Pub And Grill,63123,19,3.0,MO
9,2312 Dickerson Pike,"(None, None, u'none', None, None, None, None, ...",bBDDEgkFA1Otx9Lfe7BZUQ,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",Nashville,"(6:0-16:0, 0:0-0:0, 6:0-17:0, 6:0-21:0, 6:0-16...",1,36.208102,-86.768170,Sonic Drive-In,37207,10,1.5,TN


In [18]:
business_pd['clothes']=clothes
business_clothes=business_pd[business_pd['clothes']==1]
business_clothes.to_csv('yelp_business_clothes.csv')

In [19]:
business_clothes.shape


(5739, 15)

In [20]:

business_clothes.head(1000)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state,clothes
2,5255 E Broadway Blvd,"(None, None, None, None, None, None, None, Tru...",tUFrWirKiKi_TAnsVWINQQ,"Department Stores, Shopping, Fashion, Home & G...",Tucson,"(8:0-23:0, 8:0-22:0, 8:0-23:0, 8:0-22:0, 8:0-2...",0,32.223236,-110.880452,Target,85711,22,3.5,AZ,1
6,"8522 Eager Road, Dierbergs Brentwood Point","(None, None, None, None, None, None, None, Tru...",n_0UpQx1hsNbnPUSlodU8w,"Sporting Goods, Fashion, Shoe Stores, Shopping...",Brentwood,"(10:0-18:0, 0:0-0:0, 10:0-18:0, 12:0-18:0, 10:...",1,38.627695,-90.340465,Famous Footwear,63144,13,2.5,MO,1
10,21705 Village Lakes Sc Dr,"(None, None, None, None, None, None, None, Tru...",UJsufbvfyfONHeWdvAHKjA,"Department Stores, Shopping, Fashion",Land O' Lakes,"(9:30-21:30, 9:30-21:30, 9:30-21:30, 10:0-20:0...",1,28.190459,-82.457380,Marshalls,34639,6,3.5,FL,1
26,827-833 State St,"(None, None, None, None, None, None, None, Tru...",noByYNtDLQAra9ccqxdfDw,"Women's Clothing, Accessories, Children's Clot...",Santa Barbara,"(11:0-18:0, 11:0-18:0, 11:0-18:0, 11:0-18:0, 1...",1,34.420209,-119.700460,H&M,93101,24,3.0,CA,1
32,30 W Lancaster Ave,"(None, None, None, None, None, None, None, Tru...",8sshLb4UU7emeUDvtJWnpA,"Dance Wear, Sports Wear, Children's Clothing, ...",Paoli,"(10:0-19:0, 0:0-0:0, 10:0-17:0, 12:0-15:0, 10:...",1,40.041585,-75.484953,DanceLine,19301,11,4.5,PA,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,707 Neshaminy Mall,"(None, None, None, None, None, None, None, Non...",wYxfFGvg2WNFhRye7Ys_NQ,"Customized Merchandise, Beauty & Spas, Fashion...",Bensalem,"(10:0-21:0, 10:0-21:0, 10:0-21:0, 11:0-18:0, 1...",1,40.139690,-74.955694,Way Out,19020,5,3.5,PA,1
26010,6865 Sierra Center Pkwy,"(None, None, None, None, None, None, None, Tru...",8FVTzAzSrvVbcgXqJyKbqQ,"Used, Vintage & Consignment, Shopping, Office ...",Reno,"(9:0-20:0, 9:0-17:0, 9:0-20:0, 10:0-15:0, 9:0-...",1,39.463732,-119.779564,Rocks Consign Furniture,89511,118,4.0,NV,1
26054,1501 W 86th St,"(None, None, None, None, None, None, None, Fal...",Rrx1bkPopx4LQMiNOzLu9g,"Used, Vintage & Consignment, Fashion, Thrift S...",Indianapolis,"(10:0-18:0, 10:0-18:0, 10:0-18:0, None, 10:0-1...",1,39.911346,-86.186912,Thrifty Threads,46260,30,4.0,IN,1
26055,5015 - 111 Street NW,"(None, None, None, None, None, None, None, Tru...",dDDAvlEYMRmtjRy9qklh0g,"Women's Clothing, Fashion, Men's Clothing, Sho...",Edmonton,None,1,53.484696,-113.515212,Zara,T6H 4M6,11,3.0,AB,1


In [21]:
business_clothes_ids=business_clothes['business_id']

In [22]:
# From the review dataset we will extract the reviews for food stores
business_clothes_reviews=reviews_pd[reviews_pd['business_id'].isin(business_clothes_ids)]
business_clothes_reviews.to_csv('yelp_business_clothes_reviews.csv')
business_clothes_reviews.shape

(12169, 9)

In [24]:
# Food store Reviews
business_clothes_reviews.head(10)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
133,1wp5t7VQsRSj2qcVDPjzWw,0,2016-08-30 20:41:40,0,vfEdz1-fZu2TgdtqjnVJbQ,3.0,I came here because of the higher rating and t...,6,8oGROLZb_JjaQ11pdDEaTQ
209,5_sby6qh9I6rwnSuoURSaQ,0,2015-04-28 22:05:30,0,UFqNsPYAQPI95oMb4WEU2A,3.0,Love this store especially with the nice selec...,1,Rrw53p4DNkj15djU1_hXHQ
380,b0hmAWiptSBtvUgGhqfsMA,0,2015-06-21 05:23:31,0,Bay4m4pdurbk3m4rs_2Niw,5.0,I fell in love with this boutique the second I...,0,yOhY34q6jKhPvk6EUAPgPw
391,7gfuvFznz4_vFDH06CqvBA,0,2010-05-14 14:52:36,0,IoFS0qPiXFVaZNQHq6bGGQ,4.0,"Yuppers, I'm a fan of Old Navy. I can outfit m...",0,bcCHgqnuRQuOchZQF7NGeQ
542,X9fFZcUErLMZtisQI9qx1Q,1,2014-11-12 17:36:00,0,QWh5NnOiI0XH92NSJpgQ6g,5.0,This review is for the Louis Vuitton store in ...,2,yd1xDWV-WxirMDQVsrFzdw
551,cAUe8LFnDjii_sx_4fRMaA,1,2016-05-07 19:54:06,0,EPeaaGQnE2V3awlw-XSqAA,5.0,Once Upon A Child is an AWESOME store! With q...,0,8BONrcRPZiPuuXPca1r22g
573,q3pBR16m1-w2y1fNpRDv5w,0,2014-06-29 14:32:20,0,XnC04UaxtOKdft5c8SFQbA,4.0,Kohl's makes returns a breeze. I am a fan of ...,0,jswWHPctnZGH3qqvN4mzoQ
802,rQ2CSHRkZSn5WCu7OfVI-Q,0,2014-11-30 15:04:44,0,oL9DmLVqMiUzoZz1bA8RTQ,5.0,Went here a few weeks ago for contacts. I had ...,0,FMYrjMfUEJ9k73igiNJkuA
843,80uB-BqbkWaJgB4SLt-GhQ,0,2016-01-17 19:17:44,1,xyAYTnDbf9Qd1BxG2D8wqA,1.0,I'm laughing at all the decent reviews for thi...,3,feiItwnv-Fu92pXG9IIbew
926,rT1XbnU-U-iT1sPD54s5BA,0,2018-09-14 21:50:38,0,P-XUY4meaYJbmX_MtBxVvw,5.0,I am very happy with the selection here and ev...,0,YL5dTY9iGCYOS4q_eQFm2g


In [25]:
# Overview of the dataset
print('Number of unique Reviews:',str(len(business_clothes_reviews)))
print('Number of unique Businesses:',str(len(business_clothes_reviews.business_id.unique())))
print('Number of unique Users that posted a Review:',str(len(business_clothes_reviews.user_id.unique())))

Number of unique Reviews: 12169
Number of unique Businesses: 3989
Number of unique Users that posted a Review: 10776


In [26]:
# Merging the store info with review  dataset
review_business_data_merged=pd.merge(business_clothes_reviews,business_clothes,how='left',on='business_id')

In [27]:
# Drop unwanted columns
review_business_data_merged=review_business_data_merged.drop(columns=['hours','attributes','useful'])
review_business_data_merged.head()

,business_id,cool,date,funny,review_id,stars_x,text,user_id,address,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars_y,state,clothes
0,1wp5t7VQsRSj2qcVDPjzWw,0,2016-08-30 20:41:40,0,vfEdz1-fZu2TgdtqjnVJbQ,3.0,I came here because of the higher rating and t...,8oGROLZb_JjaQ11pdDEaTQ,25 Music Sq E,"Eyewear & Opticians, Accessories, Shopping, Fa...",Nashville,1,36.149817,-86.792105,Music City Optical,37203,47,4.0,TN,1
1,5_sby6qh9I6rwnSuoURSaQ,0,2015-04-28 22:05:30,0,UFqNsPYAQPI95oMb4WEU2A,3.0,Love this store especially with the nice selec...,Rrw53p4DNkj15djU1_hXHQ,190 N Gulph Rd,"Shopping, Women's Clothing, Fashion, Departmen...",King Of Prussia,1,40.086806,-75.392073,Nordstrom The Plaza at King of Prussia,19406,105,3.5,PA,1
2,b0hmAWiptSBtvUgGhqfsMA,0,2015-06-21 05:23:31,0,Bay4m4pdurbk3m4rs_2Niw,5.0,I fell in love with this boutique the second I...,yOhY34q6jKhPvk6EUAPgPw,323 Chartres St,"Antiques, Shopping, Women's Clothing, Fashion,...",New Orleans,1,29.954843,-90.066438,Lost and Found,70130,41,4.5,LA,1
3,7gfuvFznz4_vFDH06CqvBA,0,2010-05-14 14:52:36,0,IoFS0qPiXFVaZNQHq6bGGQ,4.0,"Yuppers, I'm a fan of Old Navy. I can outfit m...",bcCHgqnuRQuOchZQF7NGeQ,", 13300 137th Avenue","Women's Clothing, Men's Clothing, Shopping, Fa...",Edmonton,1,53.599574,-113.550589,Old Navy,T5L 5G6,9,4.0,AB,1
4,X9fFZcUErLMZtisQI9qx1Q,1,2014-11-12 17:36:00,0,QWh5NnOiI0XH92NSJpgQ6g,5.0,This review is for the Louis Vuitton store in ...,yd1xDWV-WxirMDQVsrFzdw,10180 101 Street NW,"Men's Clothing, Accessories, Women's Clothing,...",Edmonton,0,53.542652,-113.494415,Holt Renfrew,T5J 3S4,25,3.0,AB,1


In [28]:
review_business_data_merged.to_csv('yelp_reviews_business_merged_clothes.csv')

In [29]:
review_business_data_merged

,business_id,cool,date,funny,review_id,stars_x,text,user_id,address,categories,city,is_open,latitude,longitude,name,postal_code,review_count,stars_y,state,clothes
0,1wp5t7VQsRSj2qcVDPjzWw,0,2016-08-30 20:41:40,0,vfEdz1-fZu2TgdtqjnVJbQ,3.0,I came here because of the higher rating and t...,8oGROLZb_JjaQ11pdDEaTQ,25 Music Sq E,"Eyewear & Opticians, Accessories, Shopping, Fa...",Nashville,1,36.149817,-86.792105,Music City Optical,37203,47,4.0,TN,1
1,5_sby6qh9I6rwnSuoURSaQ,0,2015-04-28 22:05:30,0,UFqNsPYAQPI95oMb4WEU2A,3.0,Love this store especially with the nice selec...,Rrw53p4DNkj15djU1_hXHQ,190 N Gulph Rd,"Shopping, Women's Clothing, Fashion, Departmen...",King Of Prussia,1,40.086806,-75.392073,Nordstrom The Plaza at King of Prussia,19406,105,3.5,PA,1
2,b0hmAWiptSBtvUgGhqfsMA,0,2015-06-21 05:23:31,0,Bay4m4pdurbk3m4rs_2Niw,5.0,I fell in love with this boutique the second I...,yOhY34q6jKhPvk6EUAPgPw,323 Chartres St,"Antiques, Shopping, Women's Clothing, Fashion,...",New Orleans,1,29.954843,-90.066438,Lost and Found,70130,41,4.5,LA,1
3,7gfuvFznz4_vFDH06CqvBA,0,2010-05-14 14:52:36,0,IoFS0qPiXFVaZNQHq6bGGQ,4.0,"Yuppers, I'm a fan of Old Navy. I can outfit m...",bcCHgqnuRQuOchZQF7NGeQ,", 13300 137th Avenue","Women's Clothing, Men's Clothing, Shopping, Fa...",Edmonton,1,53.599574,-113.550589,Old Navy,T5L 5G6,9,4.0,AB,1
4,X9fFZcUErLMZtisQI9qx1Q,1,2014-11-12 17:36:00,0,QWh5NnOiI0XH92NSJpgQ6g,5.0,This review is for the Louis Vuitton store in ...,yd1xDWV-WxirMDQVsrFzdw,10180 101 Street NW,"Men's Clothing, Accessories, Women's Clothing,...",Edmonton,0,53.542652,-113.494415,Holt Renfrew,T5J 3S4,25,3.0,AB,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12164,LUcs_N1k6DYCUUhGWE0vIg,0,2019-10-25 23:38:56,0,4wr74diQTJb6nHqkf-236w,2.0,I went to Dillard's with my 5yr old daughter t...,ZbSt0XoSgI0mZFjXGk__Qg,13933 S Virginia St,"Department Stores, Children's Clothing, Men's ...",Reno,1,39.404808,-119.752919,Dillard's,89511,84,3.5,NV,1
12165,pD5iEviqhZ9ZnsBi26lb7Q,0,2014-04-12 19:15:55,0,K7c0ISjP0mxYMlqaUyoL7Q,3.0,This studio is in a great location and has awe...,V0MstZHJ7DA5m585GkVIyw,"1701 Walnut St, Fl 4","Active Life, Barre Classes, Women's Clothing, ...",Philadelphia,1,39.950217,-75.169199,Pure Barre - Philadelphia Center City,19103,61,4.0,PA,1
12166,gvA1QnLqzt7QwuAmDyzbRA,1,2021-08-15 04:16:35,0,t0iZ7oWuKXn44GZsbERk2A,5.0,My son was looking for a new bike and has foun...,qNPfnXBEGGxtDJVZvK8fIA,1927 Sophie Wright Pl,"Food, Local Services, Women's Clothing, Fashio...",New Orleans,1,29.931537,-90.074619,Dashing Bicycles & Accessories,70130,41,4.5,LA,1
12167,DAKlJXSQHL0qor--9h8UAg,0,2021-11-11 13:19:32,1,-EICle060Zv56jTAv0QsQA,1.0,Last night I called the studio to let them kno...,z4GkZisdeNH-DnoKm9wZqA,300 Montgomery Mall,"Shopping, Fashion, Women's Clothing, Children'...",North Wales,1,40.233466,-75.244292,JCPenney,19454,18,3.0,PA,1


In [30]:
review_business_data_merged=review_business_data_merged.drop(columns=['is_open'])
review_business_data_merged.head()

,business_id,cool,date,funny,review_id,stars_x,text,user_id,address,categories,city,latitude,longitude,name,postal_code,review_count,stars_y,state,clothes
0,1wp5t7VQsRSj2qcVDPjzWw,0,2016-08-30 20:41:40,0,vfEdz1-fZu2TgdtqjnVJbQ,3.0,I came here because of the higher rating and t...,8oGROLZb_JjaQ11pdDEaTQ,25 Music Sq E,"Eyewear & Opticians, Accessories, Shopping, Fa...",Nashville,36.149817,-86.792105,Music City Optical,37203,47,4.0,TN,1
1,5_sby6qh9I6rwnSuoURSaQ,0,2015-04-28 22:05:30,0,UFqNsPYAQPI95oMb4WEU2A,3.0,Love this store especially with the nice selec...,Rrw53p4DNkj15djU1_hXHQ,190 N Gulph Rd,"Shopping, Women's Clothing, Fashion, Departmen...",King Of Prussia,40.086806,-75.392073,Nordstrom The Plaza at King of Prussia,19406,105,3.5,PA,1
2,b0hmAWiptSBtvUgGhqfsMA,0,2015-06-21 05:23:31,0,Bay4m4pdurbk3m4rs_2Niw,5.0,I fell in love with this boutique the second I...,yOhY34q6jKhPvk6EUAPgPw,323 Chartres St,"Antiques, Shopping, Women's Clothing, Fashion,...",New Orleans,29.954843,-90.066438,Lost and Found,70130,41,4.5,LA,1
3,7gfuvFznz4_vFDH06CqvBA,0,2010-05-14 14:52:36,0,IoFS0qPiXFVaZNQHq6bGGQ,4.0,"Yuppers, I'm a fan of Old Navy. I can outfit m...",bcCHgqnuRQuOchZQF7NGeQ,", 13300 137th Avenue","Women's Clothing, Men's Clothing, Shopping, Fa...",Edmonton,53.599574,-113.550589,Old Navy,T5L 5G6,9,4.0,AB,1
4,X9fFZcUErLMZtisQI9qx1Q,1,2014-11-12 17:36:00,0,QWh5NnOiI0XH92NSJpgQ6g,5.0,This review is for the Louis Vuitton store in ...,yd1xDWV-WxirMDQVsrFzdw,10180 101 Street NW,"Men's Clothing, Accessories, Women's Clothing,...",Edmonton,53.542652,-113.494415,Holt Renfrew,T5J 3S4,25,3.0,AB,1


In [31]:
review_business_data_merged.to_csv('yelp_reviews_business_merged_clothes.csv')

In [32]:
review_business_data_merged=pd.read_csv('yelp_reviews_business_merged_clothes.csv')
